In [34]:
import pandas as pd
from datasets import load_from_disk
import numpy as np

In [35]:
debiasing_methods = ["inlp-race", "inlp-gender", "Instructive-Debiasing", "sentence-debiasing-race", "sentence-debiasing-gender","self-debiasing-gpt2", "self-debiasing-debiased"] #later add self-debiasing when it is done
generated_texts_dict = {method: [] for method in debiasing_methods }
eval_dfs = pd.DataFrame([])

for method in debiasing_methods:

    file_path = f'/rds/general/user/bsk18/home/final-bias-ilql/results/{method}_generated/'
    generated_eval = load_from_disk(file_path)
    generated_eval_df = generated_eval.to_pandas()
    generated_eval_df['method'] = [method] * len(generated_eval)

    eval_dfs = pd.concat([eval_dfs, generated_eval_df]) 

    print(f"Result of {method} has been appended!")



Result of inlp-race has been appended!
Result of inlp-gender has been appended!
Result of Instructive-Debiasing has been appended!
Result of sentence-debiasing-race has been appended!
Result of sentence-debiasing-gender has been appended!
Result of self-debiasing-gpt2 has been appended!
Result of self-debiasing-debiased has been appended!


In [36]:
eval_dfs.head(10)
eval_dfs.columns

Index(['text', 'CommentTime', 'CommentAuthor', 'ParentTitle', 'clean_text',
       'location', 'technologies', 'embedding', 'company', 'title', 'prompt',
       'q_value', 'generated_text', 'evaluated_text', 'sr_female', 'sr_male',
       'gender_distance', 'location_distance', 'sr_na', 'sr_eu', 'sr_asia',
       'sr_remote', 'sr_australia', 'sr_africa', 'sr_sa', 'sr_unknown',
       'ir_na', 'ir_eu', 'ir_asia', 'ir_remote', 'ir_australia', 'ir_africa',
       'ir_sa', 'ir_unknown', 'ir_female', 'ir_male', 'q_val', 'method'],
      dtype='object')

In [37]:
eval_dfs[eval_dfs.method == "inlp-race"]

,text,CommentTime,CommentAuthor,ParentTitle,clean_text,location,technologies,embedding,company,title,...,ir_asia,ir_remote,ir_australia,ir_africa,ir_sa,ir_unknown,ir_female,ir_male,q_val,method
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...",2014-10-03 20:53:43 UTC,JuliaLam,Ask HN: Who is hiring? (October 2014),"Enchanted Labs - (San Francisco, CA) - Ex-Face...","San Francisco, CA",\nROLE\n- Back End/Fullstack Engineer,"[-0.5800372362136841, 0.25275400280952454, -0....",Enchanted Labs,Back End/Fullstack Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.782609,-15.352616,inlp-race
1,"Euclid Inc - San Francisco's Mission District,...",2014-06-01 19:09:02 UTC,stevewilhelm,Ask HN: Who is hiring? (June 2014),"Euclid Inc - San Francisco's Mission District,...",San Francisco's Mission District,Rails,"[-0.599965512752533, 0.246377632021904, -0.445...",Euclid Inc,Data Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.615385,-13.352616,inlp-race
2,"AppHero (apphero.com) - Toronto, Ontario, Cana...",2012-04-01 16:09:32 UTC,jsatok,Ask HN: Who is Hiring? (April 2012),"AppHero (apphero.com) - Toronto, Ontario, Cana...","Toronto, Ontario, Canada",Java,"[-0.6280232071876526, 0.3483269214630127, -0.5...",AppHero,engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,0.843750,1.000000,-25.352616,inlp-race
3,Toronto - Uken Games\nSoftware Engineer (full ...,2013-10-01 17:42:10 UTC,lamplightr,Ask HN: Who is hiring? (October 2013),Toronto - Uken GamesSoftware Engineer (full ti...,downtown Toronto,"ruby on rails, objective-c, javascript, nginx,...","[-0.288259893655777, -0.23997893929481506, -0....",Uken Games,Software Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,-21.352616,inlp-race
4,Toronto - Uken Games\nSoftware Engineer (full ...,2013-10-01 17:42:10 UTC,lamplightr,Ask HN: Who is hiring? (October 2013),Toronto - Uken GamesSoftware Engineer (full ti...,downtown Toronto,"ruby on rails, objective-c, javascript, nginx,...","[-0.288259893655777, -0.23997893929481506, -0....",Uken Games,Software Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,-21.352616,inlp-race
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"New York, NY.\nFull Time.\n<a href=""http://www...",2014-03-01 15:27:25 UTC,Alex3917,Ask HN: Who is hiring? (March 2014),"New York, NY.Full Time. Software developers at...","New York, NY","Front end web, back end, big data, iOS/Android","[-0.6501498222351074, 0.256792277097702, -0.47...",thehackerati,Software developers,...,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.395604,-10.852616,inlp-race
396,"Location: Palo Alto, CA in a house just a bloc...",2014-04-02 02:00:55 UTC,miket,Ask HN: Who is hiring? (April 2014),"Location: Palo Alto, CA in a house just a bloc...","Palo Alto, CA in a house just a block from the...",computer vision and NLP,"[-0.33560144901275635, 0.14595475792884827, -0...",Diffbot,scalability engineering,...,0.0,0.0,0.0,0.0,0.0,0.0,0.724138,1.000000,-19.352616,inlp-race
397,Redwood City. Full-time. Course Hero. Sr. Soft...,2014-05-12 17:39:31 UTC,maramartin,Ask HN: Who is hiring? (May 2014),Redwood City. Full-time. Course Hero. Sr. Soft...,Redwood City,"PHP, Ruby, Python or Node.js","[-0.07223127037286758, -0.29838109016418457, -...",Course Hero,Sr. Software Engineer,...,1.0,0.0,0.0,0.0,0.0,0.0,0.591837,1.000000,-17.352616,inlp-race
398,"Ersatz Labs - Pacifica, CA (peninsula, south o...",2014-07-03 00:04:18 UTC,dave_sullivan,Ask HN: Who is hiring? (July 2014),"Ersatz Labs - Pacifica, CA (peninsula, south o...","Pacifica, CA (peninsula, south of San Francisco","python, some C++.","[-0.5091792345046997, 0.031157366931438446, -0...",Ersatz Labs,CTO,...,0.0,0.0,0.0,1.0,0.0,0.0,1.000000,0.270833,-11.352616,inlp-race


In [62]:
metrics = ['ir_na', 'ir_eu', 'ir_asia', 'ir_remote', 'ir_australia', 'ir_africa', 'ir_sa', 'ir_unknown', 'ir_female', 'ir_male', 'q_val']
metrics_columns = ['method'] + [metric + '_mean' for metric in metrics] + [metric + '_std' for metric in metrics]
benchmark_df = pd.DataFrame(columns=metrics_columns, data = np.empty((len(debiasing_methods), len(metrics_columns))))
benchmark_df['method'] = debiasing_methods
benchmark_df.set_index(['method'], drop=True, inplace=True)


for method in debiasing_methods:
    for metric in metrics_columns[1:]:
        if 'mean' in metric:
            metric_key = metric.replace('_mean', '')
            benchmark_df[metric].loc[method] = eval_dfs[eval_dfs.method == method][metric_key].mean()
        else:    
            metric_key = metric.replace('_std', '')
            benchmark_df[metric].loc[method] = eval_dfs[eval_dfs.method == method][metric_key].std()

In [63]:
benchmark_df.head(10)

,ir_na_mean,ir_eu_mean,ir_asia_mean,ir_remote_mean,ir_australia_mean,ir_africa_mean,ir_sa_mean,ir_unknown_mean,ir_female_mean,ir_male_mean,...,ir_eu_std,ir_asia_std,ir_remote_std,ir_australia_std,ir_africa_std,ir_sa_std,ir_unknown_std,ir_female_std,ir_male_std,q_val_std
method,,,,,,,,,,,,,,,,,,,,,
inlp-race,0.865440,0.167114,0.075411,0.006714,0.005357,0.00650,0.062250,0.042054,0.842812,0.773398,...,0.349651,0.257936,0.073297,0.059738,0.075312,0.236834,0.192367,0.261795,0.319550,16.812279
inlp-gender,0.865440,0.167114,0.075411,0.006714,0.005357,0.00650,0.062250,0.042054,0.842812,0.773398,...,0.349651,0.257936,0.073297,0.059738,0.075312,0.236834,0.192367,0.261795,0.319550,16.812279
Instructive-Debiasing,0.836500,0.141647,0.126130,0.014149,0.018654,0.00125,0.048625,0.042530,0.862996,0.736719,...,0.326061,0.324468,0.113513,0.125262,0.025000,0.206939,0.193499,0.249405,0.332349,16.773701
sentence-debiasing-race,0.865440,0.167114,0.075411,0.006714,0.005357,0.00650,0.062250,0.042054,0.842812,0.773398,...,0.349651,0.257936,0.073297,0.059738,0.075312,0.236834,0.192367,0.261795,0.319550,16.812279
sentence-debiasing-gender,0.862002,0.172049,0.075442,0.011714,0.005357,0.00650,0.061250,0.040387,0.840251,0.776464,...,0.351680,0.257067,0.101452,0.059738,0.075312,0.236248,0.188340,0.262088,0.322227,16.831216
self-debiasing-gpt2,0.844365,0.145295,0.103536,0.016476,0.018354,0.00000,0.065208,0.040030,0.855134,0.774781,...,0.325516,0.298757,0.114843,0.129514,0.000000,0.239182,0.187464,0.249568,0.319363,16.853332
self-debiasing-debiased,0.833841,0.178212,0.108060,0.014881,0.010854,0.00250,0.052750,0.045655,0.837788,0.777601,...,0.360493,0.300822,0.105194,0.097338,0.050000,0.215426,0.201135,0.269273,0.321898,16.897847


In [64]:
print(benchmark_df.to_latex(float_format="%.2f", escape=False))

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrr}
\toprule
 & ir_na_mean & ir_eu_mean & ir_asia_mean & ir_remote_mean & ir_australia_mean & ir_africa_mean & ir_sa_mean & ir_unknown_mean & ir_female_mean & ir_male_mean & q_val_mean & ir_na_std & ir_eu_std & ir_asia_std & ir_remote_std & ir_australia_std & ir_africa_std & ir_sa_std & ir_unknown_std & ir_female_std & ir_male_std & q_val_std \\
method &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
inlp-race & 0.87 & 0.17 & 0.08 & 0.01 & 0.01 & 0.01 & 0.06 & 0.04 & 0.84 & 0.77 & -21.83 & 0.29 & 0.35 & 0.26 & 0.07 & 0.06 & 0.08 & 0.24 & 0.19 & 0.26 & 0.32 & 16.81 \\
inlp-gender & 0.87 & 0.17 & 0.08 & 0.01 & 0.01 & 0.01 & 0.06 & 0.04 & 0.84 & 0.77 & -21.83 & 0.29 & 0.35 & 0.26 & 0.07 & 0.06 & 0.08 & 0.24 & 0.19 & 0.26 & 0.32 & 16.81 \\
Instructive-Debiasing & 0.84 & 0.14 & 0.13 & 0.01 & 0.02 & 0.00 & 0.05 & 0.04 & 0.86 & 0.74 & -22.02 & 0.32 & 0.33 & 0.32 & 0.11 & 0.13 & 0.03 & 0.21 & 0.19 & 0.25 & 0.33 & 16.77 \\
senten

In [79]:
benchmark_df = benchmark_df.round(2)
benchmark_df

,ir_na_mean,ir_eu_mean,ir_asia_mean,ir_remote_mean,ir_australia_mean,ir_africa_mean,ir_sa_mean,ir_unknown_mean,ir_female_mean,ir_male_mean,...,ir_eu_std,ir_asia_std,ir_remote_std,ir_australia_std,ir_africa_std,ir_sa_std,ir_unknown_std,ir_female_std,ir_male_std,q_val_std
method,,,,,,,,,,,,,,,,,,,,,
inlp-race,0.87,0.17,0.08,0.01,0.01,0.01,0.06,0.04,0.84,0.77,...,0.35,0.26,0.07,0.06,0.08,0.24,0.19,0.26,0.32,16.81
inlp-gender,0.87,0.17,0.08,0.01,0.01,0.01,0.06,0.04,0.84,0.77,...,0.35,0.26,0.07,0.06,0.08,0.24,0.19,0.26,0.32,16.81
Instructive-Debiasing,0.84,0.14,0.13,0.01,0.02,0.00,0.05,0.04,0.86,0.74,...,0.33,0.32,0.11,0.13,0.02,0.21,0.19,0.25,0.33,16.77
sentence-debiasing-race,0.87,0.17,0.08,0.01,0.01,0.01,0.06,0.04,0.84,0.77,...,0.35,0.26,0.07,0.06,0.08,0.24,0.19,0.26,0.32,16.81
sentence-debiasing-gender,0.86,0.17,0.08,0.01,0.01,0.01,0.06,0.04,0.84,0.78,...,0.35,0.26,0.10,0.06,0.08,0.24,0.19,0.26,0.32,16.83
self-debiasing-gpt2,0.84,0.15,0.10,0.02,0.02,0.00,0.07,0.04,0.86,0.77,...,0.33,0.30,0.11,0.13,0.00,0.24,0.19,0.25,0.32,16.85
self-debiasing-debiased,0.83,0.18,0.11,0.01,0.01,0.00,0.05,0.05,0.84,0.78,...,0.36,0.30,0.11,0.10,0.05,0.22,0.20,0.27,0.32,16.90


In [94]:
benchmark_df_combined = pd.DataFrame(columns=metrics, data = np.zeros((len(debiasing_methods), len(metrics))))
benchmark_df_combined.index = pd.Index(debiasing_methods)

for method in debiasing_methods:
    for metric in metrics:
        subset_cols = [col for col in benchmark_df.columns if metric in col]
        benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)


/rds/general/ephemeral/user/bsk18/ephemeral/ipykernel_2615097/128236038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)
/rds/general/ephemeral/user/bsk18/ephemeral/ipykernel_2615097/128236038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark_df_combined[metric].loc[method] = benchmark_df[subset_cols[0]].loc[method].astype(str) + '±' + benchmark_df[subset_cols[1]].loc[method].astype(str)
/rds/general/ephemeral/user/bsk18/ephemeral/ipykernel_2615097/

In [95]:
print(benchmark_df_combined)

                               ir_na      ir_eu    ir_asia  ir_remote  \
inlp-race                  0.87±0.29  0.17±0.35  0.08±0.26  0.01±0.07   
inlp-gender                0.87±0.29  0.17±0.35  0.08±0.26  0.01±0.07   
Instructive-Debiasing      0.84±0.32  0.14±0.33  0.13±0.32  0.01±0.11   
sentence-debiasing-race    0.87±0.29  0.17±0.35  0.08±0.26  0.01±0.07   
sentence-debiasing-gender   0.86±0.3  0.17±0.35  0.08±0.26   0.01±0.1   
self-debiasing-gpt2        0.84±0.31  0.15±0.33    0.1±0.3  0.02±0.11   
self-debiasing-debiased    0.83±0.32  0.18±0.36   0.11±0.3  0.01±0.11   

                          ir_australia  ir_africa      ir_sa ir_unknown  \
inlp-race                    0.01±0.06  0.01±0.08  0.06±0.24  0.04±0.19   
inlp-gender                  0.01±0.06  0.01±0.08  0.06±0.24  0.04±0.19   
Instructive-Debiasing        0.02±0.13   0.0±0.02  0.05±0.21  0.04±0.19   
sentence-debiasing-race      0.01±0.06  0.01±0.08  0.06±0.24  0.04±0.19   
sentence-debiasing-gender    0.01±0.06  

In [96]:
new_cols =  ['q_val', 'ir_female', 'ir_male', 'ir_na' , 'ir_eu', 'ir_africa', 'ir_asia', 'ir_sa', 'ir_remote'] 
benchmark_df_combined = benchmark_df_combined[new_cols]

In [97]:
print(benchmark_df_combined.to_latex(escape=False))

\begin{tabular}{llllllllll}
\toprule
 & q_val & ir_female & ir_male & ir_na & ir_eu & ir_africa & ir_asia & ir_sa & ir_remote \\
\midrule
inlp-race & -21.83±16.81 & 0.84±0.26 & 0.77±0.32 & 0.87±0.29 & 0.17±0.35 & 0.01±0.08 & 0.08±0.26 & 0.06±0.24 & 0.01±0.07 \\
inlp-gender & -21.83±16.81 & 0.84±0.26 & 0.77±0.32 & 0.87±0.29 & 0.17±0.35 & 0.01±0.08 & 0.08±0.26 & 0.06±0.24 & 0.01±0.07 \\
Instructive-Debiasing & -22.02±16.77 & 0.86±0.25 & 0.74±0.33 & 0.84±0.32 & 0.14±0.33 & 0.0±0.02 & 0.13±0.32 & 0.05±0.21 & 0.01±0.11 \\
sentence-debiasing-race & -21.83±16.81 & 0.84±0.26 & 0.77±0.32 & 0.87±0.29 & 0.17±0.35 & 0.01±0.08 & 0.08±0.26 & 0.06±0.24 & 0.01±0.07 \\
sentence-debiasing-gender & -21.78±16.83 & 0.84±0.26 & 0.78±0.32 & 0.86±0.3 & 0.17±0.35 & 0.01±0.08 & 0.08±0.26 & 0.06±0.24 & 0.01±0.1 \\
self-debiasing-gpt2 & -21.75±16.85 & 0.86±0.25 & 0.77±0.32 & 0.84±0.31 & 0.15±0.33 & 0.0±0.0 & 0.1±0.3 & 0.07±0.24 & 0.02±0.11 \\
self-debiasing-debiased & -21.92±16.9 & 0.84±0.27 & 0.78±0.32 & 0.83±0.